## Deep Neural Network

Imports

In [9]:
import os
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

Load in the data

In [10]:
data_movies = pd.read_csv("data/movies.csv")
data_ratings = pd.read_csv("data/ratings.csv")

Transform the data

In [11]:
merged_dataset = pd.merge(data_movies,data_ratings, how='inner', on='movieId')

refined_dataset = merged_dataset.groupby(by=['userId','title'], as_index=False).agg({"rating":"mean"})

# Encode the userid and movie title to prepare for DNN
user_enc = LabelEncoder()
refined_dataset['user'] = user_enc.fit_transform(refined_dataset['userId'].to_numpy())
item_enc = LabelEncoder()
refined_dataset['movie'] = item_enc.fit_transform(refined_dataset['title'].to_numpy())
refined_dataset['rating'] = refined_dataset['rating'].values.astype(np.float32)

In [12]:
n_users = refined_dataset['user'].nunique()
n_movies = refined_dataset['movie'].nunique()
min_rating = min(refined_dataset['rating'])
max_rating = max(refined_dataset['rating'])

Define X and Y data

In [13]:
X = refined_dataset[['user', 'movie']].values
y = refined_dataset['rating'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=50)

n_factors = 150

X_train_array = [X_train[:, 0], X_train[:, 1]]
X_test_array = [X_test[:, 0], X_test[:, 1]]

y_train = (y_train - min_rating)/(max_rating - min_rating)
y_test = (y_test - min_rating)/(max_rating - min_rating)


### Define the layers and the model

In [14]:
user = keras.layers.Input(shape = (1,))

# Embedding layer for n_factors of users
u = keras.layers.Embedding(n_users, n_factors, embeddings_initializer = 'he_normal', embeddings_regularizer = keras.regularizers.l2(1e-6))(user)
u = keras.layers.Reshape((n_factors,))(u)

# Input layer for the movies
movie = keras.layers.Input(shape = (1,))

# Embedding layer for n_factors of movies
m = keras.layers.Embedding(n_movies, n_factors, embeddings_initializer = 'he_normal', embeddings_regularizer= keras.regularizers.l2(1e-6))(movie)
m = keras.layers.Reshape((n_factors,))(m)

# stacking up both user and movie embeddings
x = keras.layers.Concatenate()([u,m])
x = keras.layers.Dropout(0.05)(x)

# Adding a Dense layer to the architecture
x = keras.layers.Dense(32, kernel_initializer='he_normal')(x)
x = keras.layers.Activation(activation='relu')(x)
x = keras.layers.Dropout(0.05)(x)

x = keras.layers.Dense(16, kernel_initializer='he_normal')(x)
x = keras.layers.Activation(activation='relu')(x)
x = keras.layers.Dropout(0.05)(x)

# Output layer 
x = keras.layers.Dense(1)(x)
x = keras.layers.Activation(activation='linear')(x)

# Define the model
model = keras.models.Model(inputs=[user,movie], outputs=x)

# Compiling the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

print(model.summary())

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 1, 150)    │     91,500 │ input_layer_2[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 1, 150)    │  1,457,850 │ input_layer_3[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_2 (Reshape) │ (None, 150)       │          0 │ embedding_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_3 (Reshape) │ (None, 150)       │          0 │ embedding_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 300)       │          0 │ reshape_2[0][0],  │
│ (Concatenate)       │                   │            │ reshape_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 300)       │          0 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 32)        │      9,632 │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 32)        │          0 │ dense_3[0][0]     │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 32)        │          0 │ activation_3[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 16)        │        528 │ dropout_4[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 16)        │          0 │ dense_4[0][0]     │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 16)        │          0 │ activation_4[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 1)         │         17 │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_5        │ (None, 1)         │          0 │ dense_5[0][0]     │
│ (Activation)        │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,559,527 (5.95 MB)

 Trainable params: 1,559,527 (5.95 MB)

 Non-trainable params: 0 (0.00 B)

None


In [15]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.75, patience=3, min_lr=0.000001, verbose=1)

history = model.fit(x = X_train_array, y = y_train, batch_size=128, epochs=70, validation_data=(X_test_array, y_test), shuffle=True,callbacks=[reduce_lr])

Epoch 1/70
709/709 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - loss: 0.1181 - mae: 0.2570 - val_loss: 0.0381 - val_mae: 0.1493 - learning_rate: 0.0010
Epoch 2/70
709/709 ━━━━━━━━━━━━━━━━━━━━ 15s 21ms/step - loss: 0.0391 - mae: 0.1512 - val_loss: 0.0373 - val_mae: 0.1474 - learning_rate: 0.0010
Epoch 3/70
709/709 ━━━━━━━━━━━━━━━━━━━━ 14s 20ms/step - loss: 0.0350 - mae: 0.1408 - val_loss: 0.0378 - val_mae: 0.1476 - learning_rate: 0.0010
Epoch 4/70
709/709 ━━━━━━━━━━━━━━━━━━━━ 14s 19ms/step - loss: 0.0307 - mae: 0.1304 - val_loss: 0.0386 - val_mae: 0.1481 - learning_rate: 0.0010
Epoch 5/70
708/709 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0275 - mae: 0.1222
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0007500000356230885.
709/709 ━━━━━━━━━━━━━━━━━━━━ 14s 19ms/step - loss: 0.0275 - mae: 0.1222 - val_loss: 0.0399 - val_mae: 0.1493 - learning_rate: 0.0010
Epoch 6/70
709/709 ━━━━━━━━━━━━━━━━━━━━ 14s 20ms/step - loss: 0.0243 - mae: 0.1136 - val_loss: 0.0406 - val_mae: 0.1504 - learning_ra

Save the model so we don't have to train the model each time

In [17]:
model.save("data/movie_recommendation_model.keras")